In [3]:
import re
import psycopg2  # for pgsql db
import pandas as pd
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('payments.csv', 'payments'),
    ('products.csv', 'products'),
    ('sellers.csv', 'sellers'),
    ('geolocation.csv', 'geolocation'),
    ('order_items.csv', 'order_items')
]

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='sql',
    database='EcommerceDB'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = "C:/Users/Rudra/Downloads/SQL_e-Commerce_sale_Analysis/data"

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

def is_potential_datetime(series):
    """
    Check if a column contains values that match common datetime patterns.
    This function samples a few rows from the series and checks for datetime patterns.
    """
    # Define a regex pattern for common datetime formats (e.g., YYYY-MM-DD or MM/DD/YYYY)
    datetime_pattern = re.compile(r'^\d{4}-\d{2}-\d{2}|^\d{2}/\d{2}/\d{4}')
    
    # Sample a few rows (10 by default) and check if all match the pattern
    sample_size = min(10, len(series))
    sample = series.dropna().astype(str).sample(sample_size, random_state=1)
    
    return all(datetime_pattern.match(val) for val in sample)

def try_convert_to_datetime(df):
    """
    Convert columns in a DataFrame to datetime only if they appear to be datetime-like.
    """
    for col in df.columns:
        if df[col].dtype == 'object' and is_potential_datetime(df[col]):
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')  # Convert to datetime where possible
            except Exception:
                pass  # If conversion fails, leave as is
    return df

# Example of how this integrates into your workflow:
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Try to convert any potential datetime columns
    df = try_convert_to_datetime(df)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the PostgreSQL table
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO \"{table_name}\" ({', '.join(['\"' + col + '\"' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()
